In [33]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

True

In [34]:
# Looad PDFS

doc_paths = ['data/Siemens SN53ES02AE.pdf']

docs = []


for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        loader = PyPDFLoader(file_path)

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")  
    
    finally:
        os.remove(file_path)

In [35]:
docs

[Document(metadata={'source': 'data/Siemens SN53ES02AE.pdf', 'page': 0}, page_content='SN53ES02AE\nGeschirrspüler\nDE Gebrauchsanleitung\nSiemens Home Appliances\nRegister your appliance on My Siemens and\ndiscover exclusive services and offers.\n'),
 Document(metadata={'source': 'data/Siemens SN53ES02AE.pdf', 'page': 1}, page_content='de\n2\nWeitere Informationen und Erklärungen fin-\nden Sie online:\nInhaltsverzeichnis\n1 Sicherheit . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa0\xa0 4\n1.1 Allgemeine Hinweise . . . . . . . . . . . . . . . . . \xa0 4\n1.2 Bestimmungsgemäßer Ge-\nbrauch . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa0 4\n1.3 Einschränkung des Nutzer-\nkreises . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa0 4\n1.4 Sichere Installation . . . . . . . . . . . . . . . . . . . . \xa0 4\n1.5 Sicherer Gebrauch . . . . . . . . . . . . . . . . . . . . \xa0 6\n1.6 Beschädigtes Gerät

In [36]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 1000
)

document_chunks = text_splitter.split_documents(docs)

In [37]:
# Tokenize and load documents to vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [38]:
def get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user","Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [39]:
def get_conversational_rag_chain(llm):
    retriever_chain = get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """    Du bist ein Haushaltsassistent-Bot und hilfst bei Fragen rund um die Bedienung von Haushaltsgeräten. 
        Deine Hauptaufgabe besteht darin, Nutzern klare und praktische Anweisungen zu geben, die leicht verständlich und sofort umsetzbar sind. 
        Die häufigsten Anfragen betreffen die Bedienung von Waschmaschinen, Geschirrspülern, Kühlschränken, Mikrowellen und anderen technischen Geräten im Haushalt.
            {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [40]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai 

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "Wie mache ich den Fernseher an?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

Um den Fernseher einzuschalten, folge diesen einfachen Schritten:

1. **Fernbedienung verwenden**: Suche die Einschalttaste (oft mit einem Power-Symbol, einem Kreis mit einem vertikalen Strich) auf deiner Fernbedienung und drücke sie.

2. **Direkt am Gerät**: Wenn du die Fernbedienung nicht finden kannst, gibt es oft auch eine Einschalttaste direkt am Fernseher. Diese befindet sich in der Regel an der Vorderseite, an der Seite oder an der Unterseite des Geräts.

3. **Stromversorgung prüfen**: Stelle sicher, dass der Fernseher an eine funktionierende Steckdose angeschlossen ist und der Netzschalter (falls vorhanden) eingeschaltet ist.

Sollte der Fernseher nicht angehen, überprüfe, ob die Batterien der Fernbedienung noch funktionieren und ob alle Kabel richtig angeschlossen sind.

In [47]:
from datasets import Dataset
from langchain.schema import HumanMessage, AIMessage

# Define questions and ground truths
questions = [
    "Welche Sicherheitsvorkehrungen sollten beim Aufstellen des Geschirrspülers beachtet werden?",
    "Wie wird der Klarspüler im Geschirrspüler nachgefüllt?",
    "Was sind die empfohlenen Maßnahmen, um Energie und Wasser zu sparen?",
    "Welche Programme sind für stark verschmutztes Geschirr geeignet?",
    "Was ist zu tun, wenn die Wasserzulauf-Anzeige leuchtet?",
    "Welche Reiniger sollten im Geschirrspüler nicht verwendet werden?",
    "Wie wird das Siebsystem des Geschirrspülers gereinigt?",
    "Welche Schritte sind bei der ersten Inbetriebnahme des Geschirrspülers erforderlich?",
    "Wie kann die automatische Türöffnung während der Trocknungsphase aktiviert werden?",
    "Was sind die Voraussetzungen für die Nutzung der Home Connect App?"
]

ground_truths = [
    ["Beim Aufstellen des Geschirrspülers ist sicherzustellen, dass die Netzanschlussleitung nicht eingeklemmt oder beschädigt wird, und dass ein Mindestabstand von 5 cm zu Wasserleitungen eingehalten wird."],
    ["Der Klarspüler wird bis zur Markierung 'max' in den Vorratsbehälter gefüllt. Übergelaufener Klarspüler sollte aus dem Spülraum entfernt werden, um Schaumbildung zu vermeiden."],
    ["Um Energie und Wasser zu sparen, sollte das Eco-50°-Programm genutzt und das Gerät immer vollständig beladen werden. Bei geringer Beladung kann die Zusatzfunktion 'Halbe Beladung' verwendet werden."],
    ["Für stark verschmutztes Geschirr wird das Programm 'Intensiv 70°' empfohlen, das speziell für eingebrannte und hartnäckige Speisereste geeignet ist."],
    ["Wenn die Wasserzulauf-Anzeige leuchtet, sollten der Wasserhahn geöffnet, der Zulaufschlauch auf Knicke überprüft und die Siebe im Wasseranschluss gereinigt werden."],
    ["Handspülmittel und chlorhaltige Reiniger dürfen nicht verwendet werden, da sie zu Schäden am Gerät und Gesundheitsrisiken führen können."],
    ["Das Siebsystem wird gereinigt, indem es aus dem Gerät entnommen, die Siebe unter fließendem Wasser gereinigt und anschließend wieder korrekt eingesetzt werden."],
    ["Vor der ersten Nutzung müssen Spezialsalz und Klarspüler eingefüllt, die Enthärtungsanlage eingestellt und das Gerät mit einem Programm bei höchster Temperatur ohne Geschirr betrieben werden."],
    ["Die automatische Türöffnung während der Trocknungsphase kann in den Grundeinstellungen aktiviert werden, indem die Option 'o01' oder 'o02' ausgewählt wird."],
    ["Für die Nutzung der Home Connect App muss das Gerät mit einem WLAN-Heimnetzwerk verbunden sein. Die App führt Schritt für Schritt durch den Anmeldeprozess."]
]

# Initialize RAG Chain
llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Use the appropriate model, e.g., "o1-preview"
    temperature=0,
    streaming=True,
)

llm_stream = llm_stream_openai
conversation_rag_chain = get_conversational_rag_chain(llm_stream)

# Ensure vector_db is correctly initialized and returns a retriever object
retriever = vector_db.as_retriever()  # Ensure this returns a retriever instance with `get_relevant_documents`

answers = []
contexts = []

# Perform inference and context retrieval
for query in questions:
    # Prepare conversation messages
    messages = [HumanMessage(content=query)]
    response_message = ""
    for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
        response_message += chunk

    answers.append(response_message)
    
    # Retrieve relevant documents for context
    try:
        retrieved_docs = retriever.invoke(query)
        contexts.append([doc.page_content for doc in retrieved_docs])
    except AttributeError as e:
        print(f"Error retrieving documents: {e}")
        contexts.append([])  # Append an empty context list in case of failure

# Convert to dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
data

# # Convert dict to dataset
dataset = Dataset.from_dict(data)

# # Inspect dataset
print(dataset)


TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [45]:
# Rename and restructure the dataset
data_fixed = {
    "question": data["question"],  # Keep the questions as-is
    "answer": data["answer"],  # Keep the answers as-is
    "retrieved_contexts": data["contexts"],  # Rename 'contexts' to 'retrieved_contexts'
    "reference": [" ".join(gt) for gt in data["ground_truths"]],  # Convert 'ground_truths' lists to strings
}

# Create a new Dataset object
dataset_fixed = Dataset.from_dict(data_fixed)


In [48]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

# Evaluate the fixed dataset
result = evaluate(
    dataset=dataset_fixed,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

# Convert the result to a pandas DataFrame for easier inspection
df = result.to_pandas()

# Display the results
df


Evaluating:  48%|████▊     | 19/40 [00:15<00:17,  1.20it/s]


KeyboardInterrupt: 

Exception raised in Job[28]: AttributeError('KeyboardInterrupt' object has no attribute 'generations')
Exception raised in Job[35]: AssertionError(LLM is not set)
Exception raised in Job[36]: AssertionError(LLM is not set)
Exception raised in Job[37]: AssertionError(set LLM before use)
Exception raised in Job[38]: AssertionError(LLM is not set)
Exception raised in Job[39]: AssertionError(LLM is not set)
Exception raised in Job[31]: AttributeError('NoneType' object has no attribute 'generate')
Exception raised in Job[32]: AttributeError('NoneType' object has no attribute 'generate')
Exception raised in Job[24]: AttributeError('NoneType' object has no attribute 'generate')
Exception raised in Job[27]: AssertionError()
Exception raised in Job[34]: AssertionError(llm must be set to compute score)
Exception raised in Job[26]: AssertionError(llm must be set to compute score)
Exception raised in Job[30]: AssertionError(llm must be set to compute score)
